In [21]:
import cvxpy as cp
import numpy as np

In [22]:
cost = np.array([3]).reshape(-1,1)
demand = np.array([[1.5],[2],[3.5],[4],[5.5]]).reshape(-1,1)
produce = np.array([8])

In [23]:
def solve_second_stage_subproblem(produce,all_demand):

    selling_price = np.array([5]).reshape(-1,1)
    salvage_value = np.array([1]).reshape(-1,1)

    duals = np.array([])
    vs = np.array([])

    for demand in all_demand:

        sell = cp.Variable(shape=(1,1))
        salvage = cp.Variable(shape=(1,1))

        objective = cp.Minimize(-1*(selling_price.T@sell)-1*(salvage_value.T@salvage))
        constraints =  [sell+salvage<=produce,sell<=demand[0],salvage>=[0],sell>=[0]]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals = np.append(duals,dual_value)
        vs = np.append(vs,problem.value)
    

    return np.array(duals).reshape(-1,1),np.array(vs).reshape(-1,1)

In [24]:
duals,qs = solve_second_stage_subproblem(produce,demand)

In [51]:
def solve_first_stage_problem(produce,g_ks,alpha_ks,senario_prob):
    
    cost = np.array([3]).reshape(-1,1)

    produce  = cp.Variable(shape=(1,1))
    v = cp.Variable(shape=(len(senario_prob),1))

    constraints = []
    for i in range(0,len(g_ks)):
        for j in range(0,len(g_ks[i])):
            constraints.append(np.array([g_ks[i][j]]).T@produce+np.array([alpha_ks[i][j]])<=v[j])

    constraints.extend([produce>=0,v>=-1000000,produce<=max(demand)])

    objective = cp.Minimize((cost.T@produce)+(senario_prob.T@v))

    problem = cp.Problem(objective, constraints)
    problem.solve()

    print(problem.value)

    return produce.value, v.value

In [52]:
produce = np.array([3])
limit = -100000

g_ks = []
alpha_ks = []

senario = 5
senario_prob = (1/senario)*(np.ones_like(demand))

for i in range(0,10):
    duals,qs = solve_second_stage_subproblem(produce,demand)

    gks_batch = []
    alpha_ks_batch = []

    for i in range(0,senario):
        gks_batch.append(-duals[i])
        alpha_ks_batch.append(qs[i]+duals[i]@produce)
  

    g_ks.append(gks_batch)
    alpha_ks.append(alpha_ks_batch)

    produce,limit= solve_first_stage_problem(produce,g_ks,alpha_ks,senario_prob)
    print(produce)
    # print(limit)

    

-4.999999999381231
[[5.5]]
-4.199999985049242
[[3.49999999]]
-4.199999970023997
[[3.49999998]]
-4.1999999670463275
[[3.49999999]]
-4.199999970682626
[[3.49999998]]
-4.199999959378781
[[3.49999999]]
-4.199999968661086
[[3.49999998]]
-4.199999961069073
[[3.49999999]]
-4.19999996378812
[[3.49999998]]
-4.199999956567822
[[3.49999998]]


In [29]:
qs.shape

(5, 1)

In [31]:
(duals@produce).shape

(5,)

In [20]:
duals.shape

(5,)

In [37]:
j=0

In [39]:
np.array([g_ks[j]]).T@produce+np.array([alpha_ks[j]])<=-10000

array([[[[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, Fa